In [1]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import joblib
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from scipy.sparse import csr_matrix
import re

# ============================================================================
# STEP 1: Define URLFeatures Class
# ============================================================================

class URLFeatures(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, urls):
        urls = np.array(urls).reshape(-1)
        feats = np.array([
            [
                len(u),
                u.count('-'),
                u.count('@'),
                u.count('?'),
                u.count('='),
                u.count('.'),
                int(u.startswith("https")),
                int(u.count("//") > 1)
            ]
            for u in urls
        ])
        return csr_matrix(feats)

# ============================================================================
# STEP 2: Load Expert Models
# ============================================================================

print("Loading Expert Models...")

URL_MODEL_PATH = r"C:\Users\angelo\Downloads\THESIS\URL_Expert-20251210T060216Z-1-001\URL_Expert\Notebook and Model\url_expert_1.pkl"
expert_1 = joblib.load(URL_MODEL_PATH)
print("✓ Expert 1 (URL) loaded")

TEXT_MODEL_PATH = r"C:\Users\angelo\Downloads\THESIS\distilbert_phishing_model"
tokenizer = AutoTokenizer.from_pretrained(TEXT_MODEL_PATH)
expert_2 = AutoModelForSequenceClassification.from_pretrained(TEXT_MODEL_PATH)
expert_2.eval()
print("✓ Expert 2 (Text) loaded")

print("\n✅ Both expert models loaded!\n")

# ============================================================================
# STEP 3: Load and Prepare Training Data
# ============================================================================

print("Loading datasets...")

# Load URL dataset
df_url = pd.read_csv(r"C:\Users\angelo\Downloads\THESIS\Dataset_5971\url_dataset_combined_1.csv")
df_url = df_url[['url', 'label']].copy()
df_url['text'] = ""  # Add empty text column

# Load text dataset (with encoding fix)
try:
    # Try UTF-8 first
    df_text = pd.read_csv(r"C:\Users\angelo\Downloads\THESIS\Dataset_5971\Dataset_100k.csv")
except UnicodeDecodeError:
    # If UTF-8 fails, try latin-1 encoding
    print("UTF-8 failed, trying latin-1 encoding...")
    df_text = pd.read_csv(r"C:\Users\angelo\Downloads\THESIS\Dataset_5971\Dataset_100k.csv", encoding='latin-1')
except:
    # Last resort: try ISO-8859-1
    print("Trying ISO-8859-1 encoding...")
    df_text = pd.read_csv(r"C:\Users\angelo\Downloads\THESIS\Dataset_5971\Dataset_100k.csv", encoding='ISO-8859-1')

df_text = df_text[['label', 'text']].copy()
df_text['url'] = ""  # Add empty url column

print(f"URL dataset: {len(df_url)} samples")
print(f"Text dataset: {len(df_text)} samples")

# Combine datasets
df_combined = pd.concat([df_url, df_text], ignore_index=True)
df_combined = df_combined.dropna()

# Use 50k samples - sweet spot between speed and accuracy
df_combined = df_combined.sample(n=min(50000, len(df_combined)), random_state=42)

print(f"Combined dataset: {len(df_combined)} samples")
print(f"Label distribution:\n{df_combined['label'].value_counts()}\n")

# Split into train and test
X = df_combined[['text', 'url']]
y = df_combined['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples\n")

# ============================================================================
# STEP 4: Helper Functions
# ============================================================================

def preprocess_text(text):
    if pd.isna(text) or text == "":
        return ""
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def calculate_phrase_score(text, phrase_dict):
    if not text:
        return 0.0
    text_lower = text.lower()
    score = 0.0
    for phrase, weight in phrase_dict.items():
        if phrase in text_lower:
            score += weight
    return min(score, 1.0)

def extract_gating_features(text, url, phrase_score):
    url_present = 1 if (url and not pd.isna(url) and url != "") else 0
    message_length = len(text.split()) if text else 0
    emoji_count = len(re.findall(r'[^\w\s,]', text)) if text else 0
    hashtag_count = text.count('#') if text else 0
    url_count = len(re.findall(r'http\S+', text)) if text else 0
    
    if text and len(text) > 0:
        capital_ratio = sum(1 for c in text if c.isupper()) / len(text)
    else:
        capital_ratio = 0.0
    
    embedding_summary = 0.0  # Placeholder
    
    features = np.array([
        url_present,
        phrase_score,
        message_length,
        emoji_count,
        hashtag_count,
        url_count,
        capital_ratio,
        embedding_summary
    ], dtype=np.float32)
    
    return features

phrase_dict = {
    'urgent': 0.3,
    'verify account': 0.5,
    'suspended': 0.4,
    'click here': 0.3,
    'confirm your': 0.4,
    'congratulations': 0.3,
    'winner': 0.4,
    'limited time': 0.3,
    'act now': 0.3,
    'security alert': 0.5,
    'claim': 0.3,
    'prize': 0.3,
    'free': 0.2,
    'bonus': 0.2,
}

# ============================================================================
# STEP 5: Define Gating Network
# ============================================================================

class GatingNetwork(nn.Module):
    def __init__(self, input_size=8, hidden_size=64, num_experts=2):
        super(GatingNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_experts)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        weights = self.softmax(x)
        return weights

# ============================================================================
# STEP 6: Train Gating Network
# ============================================================================

def train_gating_network(X_train, y_train, expert_1, expert_2, phrase_dict, num_epochs=5):
    gating_net = GatingNetwork(input_size=8, hidden_size=64, num_experts=2)
    optimizer = optim.Adam(gating_net.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    
    gating_net.train()
    
    print("Training Gating Network...")
    print("=" * 70)
    
    for epoch in range(num_epochs):
        total_loss = 0
        correct = 0
        total = 0
        
        # Shuffle data each epoch
        indices = np.random.permutation(len(X_train))
        
        for i, idx in enumerate(indices):
            text = X_train.iloc[idx]['text']
            url = X_train.iloc[idx]['url']
            label = y_train.iloc[idx]
            
            text = preprocess_text(text)
            phrase_score = calculate_phrase_score(text, phrase_dict)
            
            # Get URL expert prediction
            if url and not pd.isna(url) and url != "":
                try:
                    url_df = pd.DataFrame({'url': [url]})
                    url_probs = expert_1.predict_proba(url_df)[0]
                except:
                    url_probs = np.array([0.5, 0.5])
            else:
                url_probs = np.array([0.5, 0.5])
            
            # Get text expert prediction
            if text:
                try:
                    inputs = tokenizer(text, return_tensors='pt', padding=True, 
                                     truncation=True, max_length=128)
                    with torch.no_grad():
                        outputs = expert_2(**inputs)
                        text_probs = torch.softmax(outputs.logits, dim=1)[0].numpy()
                except:
                    text_probs = np.array([0.5, 0.5])
            else:
                text_probs = np.array([0.5, 0.5])
            
            # Convert expert predictions to tensors (no gradient needed)
            url_probs_tensor = torch.FloatTensor(url_probs).unsqueeze(0)
            text_probs_tensor = torch.FloatTensor(text_probs).unsqueeze(0)
            
            # Get gating features
            gating_features = extract_gating_features(text, url, phrase_score)
            gating_input = torch.FloatTensor(gating_features).unsqueeze(0)
            
            # Get expert weights (requires grad)
            expert_weights = gating_net(gating_input)
            
            # Combine predictions using matrix multiplication to preserve gradients
            # Stack expert predictions [batch, num_experts, num_classes]
            expert_preds = torch.stack([url_probs_tensor, text_probs_tensor], dim=1)
            
            # Expand weights for multiplication [batch, num_experts, 1]
            weights_expanded = expert_weights.unsqueeze(2)
            
            # Weighted sum: [batch, num_classes]
            final_probs = (expert_preds * weights_expanded).sum(dim=1)
            
            # Calculate loss
            label_tensor = torch.LongTensor([label])
            loss = criterion(final_probs, label_tensor)
            
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            
            # Calculate accuracy
            pred = torch.argmax(final_probs, dim=1)
            correct += (pred == label_tensor).sum().item()
            total += 1
            
            # Print progress every 1000 samples
            if (i + 1) % 1000 == 0:
                print(f"  Epoch {epoch+1}/{num_epochs} - Sample {i+1}/{len(X_train)} - Loss: {loss.item():.4f}")
        
        avg_loss = total_loss / len(X_train)
        accuracy = correct / total
        print(f"✓ Epoch {epoch+1}/{num_epochs} Complete - Avg Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")
        print("-" * 70)
    
    print("\n✅ Gating Network Training Complete!\n")
    return gating_net

# Train the gating network
gating_net = train_gating_network(X_train, y_train, expert_1, expert_2, phrase_dict, num_epochs=5)

# ============================================================================
# STEP 7: Evaluation Function
# ============================================================================

def evaluate_moe(X_test, y_test, expert_1, expert_2, gating_net, phrase_dict):
    print("Evaluating Mixture of Experts...")
    
    predictions = []
    confidences = []
    
    gating_net.eval()
    
    for idx in range(len(X_test)):
        text = X_test.iloc[idx]['text']
        url = X_test.iloc[idx]['url']
        
        text = preprocess_text(text)
        phrase_score = calculate_phrase_score(text, phrase_dict)
        
        # Get predictions
        if url and not pd.isna(url) and url != "":
            try:
                url_df = pd.DataFrame({'url': [url]})
                url_probs = expert_1.predict_proba(url_df)[0]
            except:
                url_probs = np.array([0.5, 0.5])
        else:
            url_probs = np.array([0.5, 0.5])
        
        if text:
            try:
                inputs = tokenizer(text, return_tensors='pt', padding=True, 
                                 truncation=True, max_length=128)
                with torch.no_grad():
                    outputs = expert_2(**inputs)
                    text_probs = torch.softmax(outputs.logits, dim=1)[0].numpy()
            except:
                text_probs = np.array([0.5, 0.5])
        else:
            text_probs = np.array([0.5, 0.5])
        
        gating_features = extract_gating_features(text, url, phrase_score)
        gating_input = torch.FloatTensor(gating_features).unsqueeze(0)
        
        with torch.no_grad():
            expert_weights = gating_net(gating_input)
        
        final_probs = (expert_weights[0, 0].item() * url_probs + 
                      expert_weights[0, 1].item() * text_probs)
        
        predictions.append(1 if final_probs[1] > 0.5 else 0)
        confidences.append(max(final_probs))
    
    # Calculate metrics
    cm = confusion_matrix(y_test, predictions)
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions)
    recall = recall_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    roc_auc = roc_auc_score(y_test, confidences)
    pr_auc = average_precision_score(y_test, confidences)
    
    print("\n" + "=" * 70)
    print("📊 EVALUATION RESULTS")
    print("=" * 70)
    print("\n=== Confusion Matrix ===")
    print(f"TN: {cm[0,0]:6}  |  FP: {cm[0,1]:6}")
    print(f"FN: {cm[1,0]:6}  |  TP: {cm[1,1]:6}")
    
    print("\n=== Performance Metrics ===")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"ROC-AUC:   {roc_auc:.4f}")
    print(f"PR-AUC:    {pr_auc:.4f}")
    print("=" * 70)
    
    return predictions, confidences

# Evaluate on test set
evaluate_moe(X_test, y_test, expert_1, expert_2, gating_net, phrase_dict)

# ============================================================================
# STEP 8: Save Gating Network
# ============================================================================

print("\nSaving gating network...")
torch.save(gating_net.state_dict(), 'gating_network.pth')
print("✅ Gating network saved as 'gating_network.pth'\n")

# ============================================================================
# STEP 9: Prediction Function with Trained Gating Network
# ============================================================================

def predict_with_gating(text, url, gating_net, expert_1, expert_2, phrase_dict):
    text = preprocess_text(text)
    phrase_score = calculate_phrase_score(text, phrase_dict)
    
    # Get predictions
    if url and url.strip():
        try:
            url_df = pd.DataFrame({'url': [url]})
            url_probs = expert_1.predict_proba(url_df)[0]
        except:
            url_probs = np.array([0.5, 0.5])
    else:
        url_probs = np.array([0.5, 0.5])
    
    if text:
        try:
            inputs = tokenizer(text, return_tensors='pt', padding=True, 
                             truncation=True, max_length=128)
            with torch.no_grad():
                outputs = expert_2(**inputs)
                text_probs = torch.softmax(outputs.logits, dim=1)[0].numpy()
        except:
            text_probs = np.array([0.5, 0.5])
    else:
        text_probs = np.array([0.5, 0.5])
    
    gating_features = extract_gating_features(text, url, phrase_score)
    gating_input = torch.FloatTensor(gating_features).unsqueeze(0)
    
    gating_net.eval()
    with torch.no_grad():
        expert_weights = gating_net(gating_input)
    
    final_probs = (expert_weights[0, 0].item() * url_probs + 
                  expert_weights[0, 1].item() * text_probs)
    
    prediction = "PHISHING ⚠️" if final_probs[1] > 0.5 else "SAFE ✅"
    confidence = max(final_probs) * 100
    
    url_contrib = expert_weights[0, 0].item() * 100
    text_contrib = expert_weights[0, 1].item() * 100
    
    return {
        'prediction': prediction,
        'confidence': confidence,
        'url_weight': url_contrib,
        'text_weight': text_contrib,
        'url_prediction': 'PHISHING' if url_probs[1] > 0.5 else 'SAFE',
        'text_prediction': 'PHISHING' if text_probs[1] > 0.5 else 'SAFE',
        'phrase_score': phrase_score
    }

def test_sample(input_text):
    """Smart auto-detection for testing"""
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls = re.findall(url_pattern, input_text)
    
    if urls:
        url = urls[0]
        text = re.sub(url_pattern, '', input_text).strip()
    else:
        url = ""
        text = input_text.strip()
    
    results = predict_with_gating(text, url, gating_net, expert_1, expert_2, phrase_dict)
    
    print("=" * 70)
    print("🎯 PREDICTION RESULTS (with Trained Gating Network)")
    print("=" * 70)
    if text:
        print(f"📝 Text: {text[:80]}..." if len(text) > 80 else f"📝 Text: {text}")
    if url:
        print(f"🔗 URL: {url}")
    print("\n" + "-" * 70)
    print(f"🧠 Gating Network Weights:")
    print(f"  🌐 URL Expert:  {results['url_weight']:.1f}%")
    print(f"  📄 Text Expert: {results['text_weight']:.1f}%")
    print("-" * 70)
    print(f"🎯 FINAL: {results['prediction']}")
    print(f"📊 Confidence: {results['confidence']:.2f}%")
    print("=" * 70)
    print()
    
    return results

print("\n" + "🎉 READY TO TEST! ".center(70, "="))
print("""
Use test_sample() to test with the TRAINED gating network:

test_sample("URGENT! Click here http://paypa1.com")
test_sample("Hey, want to grab coffee?")
test_sample("http://suspicious-site.com")

The gating network now LEARNS which expert to trust! 🧠
""")
print("=" * 70)

Loading Expert Models...
✓ Expert 1 (URL) loaded
✓ Expert 2 (Text) loaded

✅ Both expert models loaded!

Loading datasets...
UTF-8 failed, trying latin-1 encoding...
URL dataset: 722802 samples
Text dataset: 150355 samples
Combined dataset: 50000 samples
Label distribution:
label
0.0    32443
1.0    17557
Name: count, dtype: int64

Training set: 40000 samples
Test set: 10000 samples

Training Gating Network...
  Epoch 1/5 - Sample 1000/40000 - Loss: 0.3143
  Epoch 1/5 - Sample 2000/40000 - Loss: 0.5039
  Epoch 1/5 - Sample 3000/40000 - Loss: 0.3153
  Epoch 1/5 - Sample 4000/40000 - Loss: 0.3133
  Epoch 1/5 - Sample 5000/40000 - Loss: 0.4942
  Epoch 1/5 - Sample 6000/40000 - Loss: 0.3156
  Epoch 1/5 - Sample 7000/40000 - Loss: 0.3154
  Epoch 1/5 - Sample 8000/40000 - Loss: 0.3459
  Epoch 1/5 - Sample 9000/40000 - Loss: 0.3135
  Epoch 1/5 - Sample 10000/40000 - Loss: 0.3472
  Epoch 1/5 - Sample 11000/40000 - Loss: 0.3182
  Epoch 1/5 - Sample 12000/40000 - Loss: 0.4932
  Epoch 1/5 - Sampl

In [2]:
# ============================================================================
# COMPLETE MODEL SAVING
# ============================================================================

import pickle

print("Saving complete MoE system...")

# 1. Save Gating Network (you already have this)
torch.save(gating_net.state_dict(), 'gating_network.pth')
print("✓ Gating network saved")

# 2. Save phrase dictionary
with open('phrase_dict.pkl', 'wb') as f:
    pickle.dump(phrase_dict, f)
print("✓ Phrase dictionary saved")

# 3. Expert models are already saved at their original locations:
# - expert_1 (URL): Already at URL_MODEL_PATH
# - expert_2 (Text): Already at TEXT_MODEL_PATH
print("✓ Expert models already saved at original locations")

print("\n✅ Complete MoE system saved!")
print("\nSaved files:")
print("  - gating_network.pth (trained gating network)")
print("  - phrase_dict.pkl (phrase scoring dictionary)")
print(f"  - {URL_MODEL_PATH} (URL expert)")
print(f"  - {TEXT_MODEL_PATH} (Text expert)")

Saving complete MoE system...
✓ Gating network saved
✓ Phrase dictionary saved
✓ Expert models already saved at original locations

✅ Complete MoE system saved!

Saved files:
  - gating_network.pth (trained gating network)
  - phrase_dict.pkl (phrase scoring dictionary)
  - C:\Users\angelo\Downloads\THESIS\URL_Expert-20251210T060216Z-1-001\URL_Expert\Notebook and Model\url_expert_1.pkl (URL expert)
  - C:\Users\angelo\Downloads\THESIS\distilbert_phishing_model (Text expert)


In [5]:
# ============================================================================
# LOADING COMPLETE MODEL
# ============================================================================

def load_moe_system():
    """Load complete trained MoE system"""
    
    # 1. Load expert models
    expert_1 = joblib.load(r"C:\Users\angelo\Downloads\THESIS\URL_Expert-20251210T060216Z-1-001\URL_Expert\Notebook and Model\url_expert_1.pkl")
    
    tokenizer = AutoTokenizer.from_pretrained(r"C:\Users\angelo\Downloads\THESIS\distilbert_phishing_model")
    expert_2 = AutoModelForSequenceClassification.from_pretrained(r"C:\Users\angelo\Downloads\THESIS\distilbert_phishing_model")
    expert_2.eval()
    
    # 2. Load gating network
    gating_net = GatingNetwork(input_size=8, hidden_size=64, num_experts=2)
    gating_net.load_state_dict(torch.load('gating_network.pth'))
    gating_net.eval()
    
    # 3. Load phrase dictionary
    with open('phrase_dict.pkl', 'rb') as f:
        phrase_dict = pickle.load(f)
    
    print("✅ Complete MoE system loaded!")
    
    return expert_1, expert_2, tokenizer, gating_net, phrase_dict

# Usage:
# expert_1, expert_2, tokenizer, gating_net, phrase_dict = load_moe_system()
# Now you can use predict_with_gating() or test_sample()